## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [5]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

import pyvista as pv
pv.start_xvfb()

# import vtkmodules.all as vtk
# render_window = vtk.vtkRenderWindow()
# render_window.SetOffScreenRendering(1)


Set paths to the file for running splats

In [6]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [10]:
splatter.mesh(overwrite=True)


Available runs:
[0] 2025-07-11_171420


[15:36:50] Auto image downscale factor of 2                                                 ]8;id=489926;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=838217;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[15:37:18] use color only optimization with sigmoid activation                                         ]8;id=105149;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=424196;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Processing frames:   0%|          | 0/441 [00:00<?, ?it/s]/tmp/ns-extension/ns_extension/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()
Processing frames:  69%|██████▉   | 306/441 [01:28<00:38,  3.46it/s]


KeyboardInterrupt: 

In [16]:
getattr(splatter.model, "normals")

tensor([[-0.2978,  0.9455, -0.1318],
        [-0.6500, -0.1055,  0.7526],
        [-0.3942,  0.1180,  0.9114],
        ...,
        [ 0.3353,  0.1452,  0.9309],
        [-0.0880,  0.8469,  0.5244],
        [ 0.6734,  0.4579, -0.5804]], device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"]
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[15:42:32] Auto image downscale factor of 2                                                 ]8;id=503835;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=354941;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[15:42:58] use color only optimization with sigmoid activation                                         ]8;id=339141;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=174535;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

In [9]:
splatter.plot_mesh(attribute=similarity)

Number of points: 133390
Number of cells: 247846
Bounds: BoundsTuple(x_min=-0.8834666609764099, x_max=0.7600077986717224, y_min=-0.12439944595098495, y_max=2.6558642387390137, z_min=-1.2267214059829712, z_max=1.1309670209884644)


Widget(value='<iframe src="http://localhost:39865/index.html?ui=P_0x790d1f7d9210_3&reconnect=auto" class="pyvi…

Plot similarity maps

In [6]:
splatter.plot_mesh() #attribute=similarity)

Number of points: 133390
Number of cells: 247846
Bounds: BoundsTuple(x_min=-0.8834666609764099, x_max=0.7600077986717224, y_min=-0.12439944595098495, y_max=2.6558642387390137, z_min=-1.2267214059829712, z_max=1.1309670209884644)


Widget(value='<iframe src="http://localhost:43885/index.html?ui=P_0x730850a38fd0_1&reconnect=auto" class="pyvi…

In [8]:
import pyvista as pv

# Load the PLY file
mesh = pv.read(splatter.config['mesh_info']['mesh'])

In [16]:
# Create a plotter and add the mesh
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars='RGB', rgb=True)
plotter.show_axes()
plotter.show()

# mesh.plot(scalars='RGB', rgb=True)

Widget(value='<iframe src="http://localhost:39865/index.html?ui=P_0x790d12b3d1e0_9&reconnect=auto" class="pyvi…